In [1]:
%matplotlib inline
import scipy.io as scio
import matplotlib.pyplot as plt
import matplotlib as mpl
import cartopy.feature as cfeature
import cartopy.crs as ccrs
import netCDF4 as nc
import numpy as np
from Load_data import Data_from_nc
import xarray as xr
import math
import gc

data_land_mask = Data_from_nc('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/surfdata_0.9x1.25_hist_78pfts_CMIP6_simyr1850_c201126.nc')
data_mask = data_land_mask.load_variable('PFTDATA_MASK')
data_mask = data_mask[29:, :]

#create func to compute PR
def comp_PR(ctl,scen,q):
    #first get quantile from ctl
    qctl=ctl.quantile(q,dim='time')
    
    del ctl
    gc.collect()
    

    #get subsample scenarios
    qscen=xr.where(scen>qctl,1,0)
    pscen=qscen.sum('time')/len(qscen.time)
    
    del qscen
    gc.collect()

    #compute PR
    PR_scen=pscen/(1-q)
    PR_scen = np.where(data_mask==0, np.nan, PR_scen)
    PR_scen = np.where(PR_scen==0, np.nan, PR_scen)
    
    return PR_scen

def get_data_from_mat(file, variable):
    var_dict = scio.loadmat(file)
    var = var_dict[variable]
    var = var[:, 29:, :]
    return var

data_surface = Data_from_nc(\
                            '/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/' + \
                            'surfdata_irrigation_method.nc'\
                           )   #load the data

data_irrigation_method = data_surface.load_variable('irrigation_method')

data_lon = data_surface.load_variable('LONGXY')
data_lon = data_lon[0,:]
data_lat = data_surface.load_variable('LATIXY')
data_lat = data_lat[29:,0]
data_lat = data_lat[:]
data_time = np.array(range(262800))

def get_data_for_this_study_1(metric, period, exp, ens):
    data_out = get_data_from_mat('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/after_process/NorESM/post_process/3_hourly/' +\
                                             metric + '_' + period + '_3_hourly_' + exp + ens + '_1.mat', metric)
    data_out = np.array(data_out)
    return data_out

def get_data_for_this_study_2(metric, period, exp, ens):
    data_out = get_data_from_mat('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/after_process/NorESM/post_process/3_hourly/' +\
                                             metric + '_' + period + '_3_hourly_' + exp + ens + '_2.mat', metric)
    data_out = np.array(data_out)
    return data_out

def get_data_for_this_study_3(metric, period, exp, ens):
    data_out = get_data_from_mat('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/after_process/NorESM/post_process/3_hourly/' +\
                                             metric + '_' + period + '_3_hourly_' + exp + ens + '_3.mat', metric)
    data_out = np.array(data_out)
    return data_out

def get_data_for_this_study_4(metric, period, exp, ens):
    data_out = get_data_from_mat('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/after_process/NorESM/post_process/3_hourly/' +\
                                             metric + '_' + period + '_3_hourly_' + exp + ens + '_4.mat', metric)
    data_out = np.array(data_out)
    return data_out

def nparray_to_xarray_for_this_study(np_array):
    data_xrray = xr.DataArray(np_array, coords={'time': data_time, 'y': data_lat, 'x': data_lon}, dims=["time", "y", "x"])
    return data_xrray
    
def calcu_PR_between_two_array(data_ref, data_new, perc):
    data_xr_ref = xr.DataArray(data_ref, coords={'time': data_time, 'y': data_lat, 'x': data_lon}, dims=["time", "y", "x"])
    data_xr_new = xr.DataArray(data_new, coords={'time': data_time, 'y': data_lat, 'x': data_lon}, dims=["time", "y", "x"])
    PR_result = comp_PR(data_xr_ref,data_xr_new, perc)
    return PR_result


def calcu_PR_6_ensembles(metric, perc):
    data_NOI01_1985_1994_1 = get_data_for_this_study_1(metric, '1985_1994', 'NOI', '01')
    data_NOI01_1985_1994_2 = get_data_for_this_study_2(metric, '1985_1994', 'NOI', '01')
    data_NOI01_1985_1994_3 = get_data_for_this_study_3(metric, '1985_1994', 'NOI', '01')
    data_NOI01_1985_1994_4 = get_data_for_this_study_4(metric, '1985_1994', 'NOI', '01')
    
    data_NOI01_1995_2004_1 = get_data_for_this_study_1(metric, '1995_2004', 'NOI', '01')
    data_NOI01_1995_2004_2 = get_data_for_this_study_2(metric, '1995_2004', 'NOI', '01')
    data_NOI01_1995_2004_3 = get_data_for_this_study_3(metric, '1995_2004', 'NOI', '01')
    data_NOI01_1995_2004_4 = get_data_for_this_study_4(metric, '1995_2004', 'NOI', '01')
    
    data_NOI01_2005_2014_1 = get_data_for_this_study_1(metric, '2005_2014', 'NOI', '01')
    data_NOI01_2005_2014_2 = get_data_for_this_study_2(metric, '2005_2014', 'NOI', '01')
    data_NOI01_2005_2014_3 = get_data_for_this_study_3(metric, '2005_2014', 'NOI', '01')
    data_NOI01_2005_2014_4 = get_data_for_this_study_4(metric, '2005_2014', 'NOI', '01')

    data_NOI02_1985_1994_1 = get_data_for_this_study_1(metric, '1985_1994', 'NOI', '02')
    data_NOI02_1985_1994_2 = get_data_for_this_study_2(metric, '1985_1994', 'NOI', '02')
    data_NOI02_1985_1994_3 = get_data_for_this_study_3(metric, '1985_1994', 'NOI', '02')
    data_NOI02_1985_1994_4 = get_data_for_this_study_4(metric, '1985_1994', 'NOI', '02')
    
    data_NOI02_1995_2004_1 = get_data_for_this_study_1(metric, '1995_2004', 'NOI', '02')
    data_NOI02_1995_2004_2 = get_data_for_this_study_2(metric, '1995_2004', 'NOI', '02')
    data_NOI02_1995_2004_3 = get_data_for_this_study_3(metric, '1995_2004', 'NOI', '02')
    data_NOI02_1995_2004_4 = get_data_for_this_study_4(metric, '1995_2004', 'NOI', '02')
    
    data_NOI02_2005_2014_1 = get_data_for_this_study_1(metric, '2005_2014', 'NOI', '02')
    data_NOI02_2005_2014_2 = get_data_for_this_study_2(metric, '2005_2014', 'NOI', '02')
    data_NOI02_2005_2014_3 = get_data_for_this_study_3(metric, '2005_2014', 'NOI', '02')
    data_NOI02_2005_2014_4 = get_data_for_this_study_4(metric, '2005_2014', 'NOI', '02')

    data_NOI03_1985_1994_1 = get_data_for_this_study_1(metric, '1985_1994', 'NOI', '03')
    data_NOI03_1985_1994_2 = get_data_for_this_study_2(metric, '1985_1994', 'NOI', '03')
    data_NOI03_1985_1994_3 = get_data_for_this_study_3(metric, '1985_1994', 'NOI', '03')
    data_NOI03_1985_1994_4 = get_data_for_this_study_4(metric, '1985_1994', 'NOI', '03')
    
    data_NOI03_1995_2004_1 = get_data_for_this_study_1(metric, '1995_2004', 'NOI', '03')
    data_NOI03_1995_2004_2 = get_data_for_this_study_2(metric, '1995_2004', 'NOI', '03')
    data_NOI03_1995_2004_3 = get_data_for_this_study_3(metric, '1995_2004', 'NOI', '03')
    data_NOI03_1995_2004_4 = get_data_for_this_study_4(metric, '1995_2004', 'NOI', '03')
    
    data_NOI03_2005_2014_1 = get_data_for_this_study_1(metric, '2005_2014', 'NOI', '03')
    data_NOI03_2005_2014_2 = get_data_for_this_study_2(metric, '2005_2014', 'NOI', '03')
    data_NOI03_2005_2014_3 = get_data_for_this_study_3(metric, '2005_2014', 'NOI', '03')
    data_NOI03_2005_2014_4 = get_data_for_this_study_4(metric, '2005_2014', 'NOI', '03')

    data_NOI_1985_2014 = np.concatenate((data_NOI01_1985_1994_1,
                                         data_NOI01_1985_1994_2,
                                         data_NOI01_1995_2004_1,
                                         data_NOI01_1995_2004_2,
                                         data_NOI01_2005_2014_1,
                                         data_NOI01_2005_2014_2,
                                         
                                         data_NOI01_1985_1994_3,
                                         data_NOI01_1985_1994_4,
                                         data_NOI01_1995_2004_3,
                                         data_NOI01_1995_2004_4,
                                         data_NOI01_2005_2014_3,
                                         data_NOI01_2005_2014_4,
                                         
                                         data_NOI02_1985_1994_1,
                                         data_NOI02_1985_1994_2,
                                         data_NOI02_1995_2004_1,
                                         data_NOI02_1995_2004_2,
                                         data_NOI02_2005_2014_1,
                                         data_NOI02_2005_2014_2,
                                         
                                         data_NOI02_1985_1994_3,
                                         data_NOI02_1985_1994_4,
                                         data_NOI02_1995_2004_3,
                                         data_NOI02_1995_2004_4,
                                         data_NOI02_2005_2014_3,
                                         data_NOI02_2005_2014_4,
                                         
                                         data_NOI03_1985_1994_1,
                                         data_NOI03_1985_1994_2,
                                         data_NOI03_1995_2004_1,
                                         data_NOI03_1995_2004_2,
                                         data_NOI03_2005_2014_1,
                                         data_NOI03_2005_2014_2,
                                        
                                         data_NOI03_1985_1994_3,
                                         data_NOI03_1985_1994_4,
                                         data_NOI03_1995_2004_3,
                                         data_NOI03_1995_2004_4,
                                         data_NOI03_2005_2014_3,
                                         data_NOI03_2005_2014_4,), axis = 0)
    
    del data_NOI01_1985_1994_1, data_NOI01_1985_1994_2, data_NOI02_1985_1994_1, data_NOI02_1985_1994_2, data_NOI03_1985_1994_1, data_NOI03_1985_1994_2
    gc.collect()
    del data_NOI01_1985_1994_3, data_NOI01_1985_1994_4, data_NOI02_1985_1994_4, data_NOI02_1985_1994_3, data_NOI03_1985_1994_3, data_NOI03_1985_1994_4
    gc.collect()
    
    del data_NOI01_1995_2004_1, data_NOI01_1995_2004_2, data_NOI02_1995_2004_1, data_NOI02_1995_2004_2, data_NOI03_1995_2004_1, data_NOI03_1995_2004_2
    gc.collect()
    del data_NOI01_1995_2004_3, data_NOI01_1995_2004_4, data_NOI02_1995_2004_3, data_NOI02_1995_2004_4, data_NOI03_1995_2004_3, data_NOI03_1995_2004_4

    
    del data_NOI01_2005_2014_1, data_NOI01_2005_2014_2, data_NOI02_2005_2014_1, data_NOI02_2005_2014_2, data_NOI03_2005_2014_1, data_NOI03_2005_2014_2
    gc.collect()
    del data_NOI01_2005_2014_3, data_NOI01_2005_2014_4, data_NOI02_2005_2014_3, data_NOI02_2005_2014_4, data_NOI03_2005_2014_3, data_NOI03_2005_2014_4
    gc.collect()
    
    data_time = np.array(range(data_NOI_1985_2014.shape[0]))
    data_xr_ref = xr.DataArray(data_NOI_1985_2014, coords={'time': data_time, 'y': data_lat, 'x': data_lon}, dims=["time", "y", "x"])

    
    del data_NOI_1985_2014
    gc.collect()
    
    qctl=data_xr_ref.quantile(perc,dim='time')
    del data_xr_ref
    gc.collect()
    
    
    
    
    data_IRR01_1985_1994_1 = get_data_for_this_study_1(metric, '1985_1994', 'IRR', '01')
    data_IRR01_1985_1994_2 = get_data_for_this_study_2(metric, '1985_1994', 'IRR', '01')
    data_IRR01_1985_1994_3 = get_data_for_this_study_3(metric, '1985_1994', 'IRR', '01')
    data_IRR01_1985_1994_4 = get_data_for_this_study_4(metric, '1985_1994', 'IRR', '01')
    
    data_IRR01_1995_2004_1 = get_data_for_this_study_1(metric, '1995_2004', 'IRR', '01')
    data_IRR01_1995_2004_2 = get_data_for_this_study_2(metric, '1995_2004', 'IRR', '01')
    data_IRR01_1995_2004_3 = get_data_for_this_study_3(metric, '1995_2004', 'IRR', '01')
    data_IRR01_1995_2004_4 = get_data_for_this_study_4(metric, '1995_2004', 'IRR', '01')
    
    data_IRR01_2005_2014_1 = get_data_for_this_study_1(metric, '2005_2014', 'IRR', '01')
    data_IRR01_2005_2014_2 = get_data_for_this_study_2(metric, '2005_2014', 'IRR', '01')
    data_IRR01_2005_2014_3 = get_data_for_this_study_3(metric, '2005_2014', 'IRR', '01')
    data_IRR01_2005_2014_4 = get_data_for_this_study_4(metric, '2005_2014', 'IRR', '01')
    
    data_IRR02_1985_1994_1 = get_data_for_this_study_1(metric, '1985_1994', 'IRR', '02')
    data_IRR02_1985_1994_2 = get_data_for_this_study_2(metric, '1985_1994', 'IRR', '02')
    data_IRR02_1985_1994_3 = get_data_for_this_study_3(metric, '1985_1994', 'IRR', '02')
    data_IRR02_1985_1994_4 = get_data_for_this_study_4(metric, '1985_1994', 'IRR', '02')
    
    data_IRR02_1995_2004_1 = get_data_for_this_study_1(metric, '1995_2004', 'IRR', '02')
    data_IRR02_1995_2004_2 = get_data_for_this_study_2(metric, '1995_2004', 'IRR', '02')
    data_IRR02_1995_2004_3 = get_data_for_this_study_3(metric, '1995_2004', 'IRR', '02')
    data_IRR02_1995_2004_4 = get_data_for_this_study_4(metric, '1995_2004', 'IRR', '02')
    
    data_IRR02_2005_2014_1 = get_data_for_this_study_1(metric, '2005_2014', 'IRR', '02')
    data_IRR02_2005_2014_2 = get_data_for_this_study_2(metric, '2005_2014', 'IRR', '02')
    data_IRR02_2005_2014_3 = get_data_for_this_study_3(metric, '2005_2014', 'IRR', '02')
    data_IRR02_2005_2014_4 = get_data_for_this_study_4(metric, '2005_2014', 'IRR', '02')
    
    data_IRR03_1985_1994_1 = get_data_for_this_study_1(metric, '1985_1994', 'IRR', '03')
    data_IRR03_1985_1994_2 = get_data_for_this_study_2(metric, '1985_1994', 'IRR', '03')
    data_IRR03_1985_1994_3 = get_data_for_this_study_3(metric, '1985_1994', 'IRR', '03')
    data_IRR03_1985_1994_4 = get_data_for_this_study_4(metric, '1985_1994', 'IRR', '03')
    
    data_IRR03_1995_2004_1 = get_data_for_this_study_1(metric, '1995_2004', 'IRR', '03')
    data_IRR03_1995_2004_2 = get_data_for_this_study_2(metric, '1995_2004', 'IRR', '03')
    data_IRR03_1995_2004_3 = get_data_for_this_study_3(metric, '1995_2004', 'IRR', '03')
    data_IRR03_1995_2004_4 = get_data_for_this_study_4(metric, '1995_2004', 'IRR', '03')
    
    data_IRR03_2005_2014_1 = get_data_for_this_study_1(metric, '2005_2014', 'IRR', '03')
    data_IRR03_2005_2014_2 = get_data_for_this_study_2(metric, '2005_2014', 'IRR', '03')
    data_IRR03_2005_2014_3 = get_data_for_this_study_3(metric, '2005_2014', 'IRR', '03')
    data_IRR03_2005_2014_4 = get_data_for_this_study_4(metric, '2005_2014', 'IRR', '03')
    

    
    
    data_IRR_1985_2014 = np.concatenate((data_IRR01_1985_1994_1,
                                         data_IRR01_1985_1994_2,
                                         data_IRR01_1995_2004_1,
                                         data_IRR01_1995_2004_2,
                                         data_IRR01_2005_2014_1,
                                         data_IRR01_2005_2014_2,
                                         
                                         data_IRR01_1985_1994_3,
                                         data_IRR01_1985_1994_4,
                                         data_IRR01_1995_2004_3,
                                         data_IRR01_1995_2004_4,
                                         data_IRR01_2005_2014_3,
                                         data_IRR01_2005_2014_4,
                                         
                                         data_IRR02_1985_1994_1,
                                         data_IRR02_1985_1994_2,
                                         data_IRR02_1995_2004_1,
                                         data_IRR02_1995_2004_2,
                                         data_IRR02_2005_2014_1,
                                         data_IRR02_2005_2014_2,
                                         
                                         data_IRR02_1985_1994_3,
                                         data_IRR02_1985_1994_4,
                                         data_IRR02_1995_2004_3,
                                         data_IRR02_1995_2004_4,
                                         data_IRR02_2005_2014_3,
                                         data_IRR02_2005_2014_4,
                                         
                                         data_IRR03_1985_1994_1,
                                         data_IRR03_1985_1994_2,
                                         data_IRR03_1995_2004_1,
                                         data_IRR03_1995_2004_2,
                                         data_IRR03_2005_2014_1,
                                         data_IRR03_2005_2014_2,
                                        
                                         data_IRR03_1985_1994_3,
                                         data_IRR03_1985_1994_4,
                                         data_IRR03_1995_2004_3,
                                         data_IRR03_1995_2004_4,
                                         data_IRR03_2005_2014_3,
                                         data_IRR03_2005_2014_4,), axis = 0)
    
    del data_IRR01_1985_1994_1, data_IRR01_1985_1994_2, data_IRR02_1985_1994_1, data_IRR02_1985_1994_2, data_IRR03_1985_1994_1, data_IRR03_1985_1994_2
    gc.collect()
    del data_IRR01_1985_1994_3, data_IRR01_1985_1994_4, data_IRR02_1985_1994_3, data_IRR02_1985_1994_4, data_IRR03_1985_1994_3, data_IRR03_1985_1994_4
    gc.collect()
    
    del data_IRR01_1995_2004_1, data_IRR01_1995_2004_2, data_IRR02_1995_2004_1, data_IRR02_1995_2004_2, data_IRR03_1995_2004_1, data_IRR03_1995_2004_2
    gc.collect()
    del data_IRR01_1995_2004_3, data_IRR01_1995_2004_4, data_IRR02_1995_2004_3, data_IRR02_1995_2004_4, data_IRR03_1995_2004_3, data_IRR03_1995_2004_4
    gc.collect()

    
    del data_IRR01_2005_2014_1, data_IRR01_2005_2014_2, data_IRR02_2005_2014_1, data_IRR02_2005_2014_2, data_IRR03_2005_2014_1, data_IRR03_2005_2014_2
    gc.collect()
    del data_IRR01_2005_2014_3, data_IRR01_2005_2014_4, data_IRR02_2005_2014_3, data_IRR02_2005_2014_4, data_IRR03_2005_2014_3, data_IRR03_2005_2014_4
    gc.collect()
    
    data_time = np.array(range(data_IRR_1985_2014.shape[0]))
    data_xr_new = xr.DataArray(data_IRR_1985_2014, coords={'time': data_time, 'y': data_lat, 'x': data_lon}, dims=["time", "y", "x"])
    
    
    del data_IRR_1985_2014
    gc.collect()
    
    
    #get subsample scenarios
    qscen=xr.where(data_xr_new>qctl,1,0)
    
    del data_xr_new
    gc.collect()
    
    pscen=qscen.sum('time')/len(qscen.time)
    
    del qscen
    gc.collect()

    #compute PR
    PR_scen=pscen/(1-perc)
    PR_scen = np.where(data_mask==0, np.nan, PR_scen)
    PR_scen = np.where(PR_scen==0, np.nan, PR_scen)
    
    PR_3 = PR_scen
    
    del PR_scen
    gc.collect()
    
    
    
    
    
        
    
    return PR_3

def plot_PR_single(ax, data_xarray, title, metric, perc):
    cmap='bwr'
    vmin=1/4
    vmax=4
    alpha=0.05
    levels = [1/4, 1/3, 1/2, 2/3, 1.5, 2, 3, 4]
    im = data_xarray.plot(ax=ax, cmap=cmap,vmin=vmin,vmax=vmax,levels=levels, extend='both', add_colorbar=False, add_labels=False)
    cb = plt.colorbar(im,fraction= 0.3, pad= 0.04, extend='both', orientation='horizontal')
    cb.ax.tick_params(labelsize=12)
    cb.set_ticks([0.25, 0.33333333, 0.5, 0.66666667, 1.5, 2, 3, 4])
    cb.set_ticklabels(["1/4", "1/3", "1/2", "2/3", "3/2", "2", "3", "4"])
    ax.coastlines(color='dimgray', linewidth=0.5)
    ax.add_feature(cfeature.OCEAN, color='lightgrey')
    ax.set_title(title, loc='right', fontsize=14)
    ax.set_title(metric + '_' + perc, loc='left', fontsize=14)
    ax.add_feature(cfeature.BORDERS, linestyle='-', linewidth=0.1)
    
def plot_PR_single_no_cb(ax, data_xarray, title, metric, perc):
    cmap='bwr'
    vmin=1/4
    vmax=4
    alpha=0.05
    levels = [1/4, 1/3, 1/2, 2/3, 1, 1.5, 2, 3, 4]
    im = data_xarray.plot(ax=ax, cmap=cmap,vmin=vmin,vmax=vmax,levels=levels, extend='both', add_colorbar=False, add_labels=False)
    #cb = plt.colorbar(im,fraction= 0.02, pad= 0.04, extend='both')
    #cb.set_ticks([0.25, 0.33333344, 0.5, 0.66666667, 1, 1.5, 2, 3, 4])
    #cb.set_ticklabels(["1/4", "1/3", "1/2", "2/3", "1", "3/2", "2", "3", "4"])
    ax.coastlines(color='dimgray', linewidth=0.5)
    ax.add_feature(cfeature.OCEAN, color='lightgrey')
    ax.set_title(title, loc='right', fontsize=10)
    ax.set_title(metric + '_' + perc, loc='left', fontsize=10)
    ax.add_feature(cfeature.BORDERS, linestyle='-', linewidth=0.1)
    
def plot_all(PR_1, PR_2, PR_3, metric, perc):
    fig = plt.figure(figsize=(15,6),dpi=300)
    proj=ccrs.PlateCarree()
    data_xr_1 = xr.DataArray(PR_1, coords={'y': data_lat, 'x': data_lon}, dims=["y", "x"])
    ax = plt.subplot(311, projection=proj, frameon=False)
    plot_PR_single(ax, data_xr_1, 'all forcings', metric, perc)
    

    data_xr_2 = xr.DataArray(PR_2, coords={'y': data_lat, 'x': data_lon}, dims=["y", "x"])
    ax = plt.subplot(312, projection=proj, frameon=False)
    plot_PR_single(ax, data_xr_2, 'other forcings', metric, perc)

    data_xr_3 = xr.DataArray(PR_3, coords={'y': data_lat, 'x': data_lon}, dims=["y", "x"])
    ax = plt.subplot(313, projection=proj, frameon=False)
    plot_PR_single(ax, data_xr_3, 'irrigation expansion', metric, perc)


    
def just_do_all(metric, perc, perc_str):
    PR_1, PR_2, PR_3 = calcu_PR_6_ensembles(metric, perc)
    plot_all(PR_1, PR_2, PR_3, metric, perc_str)
    
def just_do_all_even_more(metric, varname):
    PR_3 = calcu_PR_6_ensembles(metric, 0.99)
    
    scio.savemat("/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/after_process/NorESM/PR/PR_irr_" + \
                     metric + "_99.0%.mat", {'PR_3': PR_3})
    
    PR_6 = calcu_PR_6_ensembles(metric, 0.995)
    
    scio.savemat("/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/after_process/NorESM/PR/PR_irr_" + \
                     metric + "_99.5%.mat", {'PR_6': PR_6})
    
    PR_9 = calcu_PR_6_ensembles(metric, 0.999)
    
    scio.savemat("/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/after_process/NorESM/PR/PR_irr_" + \
                     metric + "_99.9%.mat", {'PR_9': PR_9})
    
    fig = plt.figure(figsize=(18,12),dpi=300)
    proj=ccrs.PlateCarree()
    
    
    
    
    
    data_xr_3 = xr.DataArray(PR_3, coords={'y': data_lat, 'x': data_lon}, dims=["y", "x"])
    ax = plt.subplot(337, projection=proj, frameon=False)
    plot_PR_single(ax, data_xr_3, 'irrigation expansion', varname, '99.0%')
    ax.text(0.01, 0.92, 'g', color='dimgrey', fontsize=10, transform=ax.transAxes, weight='bold')
    
    
    

    data_xr_6 = xr.DataArray(PR_6, coords={'y': data_lat, 'x': data_lon}, dims=["y", "x"])
    ax = plt.subplot(338, projection=proj, frameon=False)
    plot_PR_single(ax, data_xr_6, 'irrigation expansion', varname, '99.5%')
    ax.text(0.01, 0.92, 'h', color='dimgrey', fontsize=10, transform=ax.transAxes, weight='bold')
    
    
    
    

    data_xr_9 = xr.DataArray(PR_9, coords={'y': data_lat, 'x': data_lon}, dims=["y", "x"])
    ax = plt.subplot(339, projection=proj, frameon=False)
    plot_PR_single(ax, data_xr_9, 'irrigation expansion', varname, '99.9%')
    ax.text(0.01, 0.92, 'i', color='dimgrey', fontsize=10, transform=ax.transAxes, weight='bold')

In [ ]:
just_do_all_even_more('TSA', 'NorESM T_2m')
just_do_all_even_more('AT', 'NorESM AT')
just_do_all_even_more('DI', 'NorESM DI')
just_do_all_even_more('HUMIDEX', 'NorESM HU')
just_do_all_even_more('THIC', 'NorESM THIC')
just_do_all_even_more('THIP', 'NorESM THIP')
just_do_all_even_more('T_s_65', 'NorESM T_s_65')
just_do_all_even_more('T_s_80', 'NorESM T_s_80')
just_do_all_even_more('T_w', 'NorESM T_w')